In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import ee
import geemap

ee.Initialize()

In [3]:
import sys
sys.path.append('../')

In [12]:
from sla.prep import PreProcessor
from sla.classifier import decision_tree
from sla.cloud_shadow_mask import add_cloud_shadow
from sla.hill_shadow import add_hill_shadow

from sla.settings import ING

In [5]:
ing_id = "G718255O411666S"
start_year = ee.Number(2022)
end_year = ee.Number(2022)
doy_start = ee.Number(50)
doy_end = ee.Number(200)
cloudiness = ee.Number(50)
coverage = 50
hsboolean = 0
dem = "SRTM"
hs_start = ee.Number(50)
hs_end = ee.Number(200)

In [6]:
preprocessor = PreProcessor(
    ing_id=ing_id,
    start_year=start_year,
    end_year=end_year,
    doy_start=doy_start,
    doy_end=doy_end,
    cloudiness=cloudiness,
    coverage=coverage,
    hsboolean=hsboolean,
    dem=dem,
)

In [7]:
preprocessed = preprocessor.execute()

cloud_shadow_collection = preprocessed.map(add_cloud_shadow)

hill_shadow_collection = add_hill_shadow(
        image_collection=cloud_shadow_collection, hs_start=hs_start, hs_end=hs_end
    )
classified_collection = hill_shadow_collection.map(decision_tree)

### Export image collection to local

In [ ]:
# task = ee.batch.Export.image.toDrive(image=my_image,  # an ee.Image object.
#                                      region=my_geometry,  # an ee.Geometry object.
#                                      description='mock_export',
#                                      folder='gdrive_folder',
#                                      fileNamePrefix='mock_export',
#                                      scale=1000,
#                                      crs='EPSG:4326')

In [10]:
classified_collection.size().getInfo()

26

In [21]:
geometry = ING.filterMetadata("ID_local", "equals", ing_id)
geometry_raw = geometry.geometry()

def clip(img):
    return img.clip(geometry_raw)

In [22]:
coll = classified_collection.map(clip)

In [23]:
img = coll.first()

In [25]:
geemap.ee_export_image(img, '../exports/classifier/demo.tif', scale=30, crs='EPSG:32719', region=None, file_per_band=False)

Generating URL ...
Please wait ...
Data downloaded to /Users/gurupratap.matharu/Desktop/code/work/glaciers/sla-sca/exports/classifier/demo.tif


In [ ]:
res = geemap.ee_export_image_collection(coll, 
                                        '../exports/classifier/', 
                                        scale=30, 
                                        crs="EPSG:32719", 
                                        region=None, 
                                        file_per_band=False)

Total number of images: 26

Exporting 1/26: 2_2_20220311T142741_20220311T144201_T19GBQ.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/gurupratap.matharu/Desktop/code/work/glaciers/sla-sca/exports/classifier/2_2_20220311T142741_20220311T144201_T19GBQ.tif


Exporting 2/26: 2_2_20220311T142741_20220311T144201_T18GYV.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/gurupratap.matharu/Desktop/code/work/glaciers/sla-sca/exports/classifier/2_2_20220311T142741_20220311T144201_T18GYV.tif


Exporting 3/26: 2_2_20220324T143731_20220324T145300_T19GBQ.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/gurupratap.matharu/Desktop/code/work/glaciers/sla-sca/exports/classifier/2_2_20220324T143731_20220324T145300_T19GBQ.tif


Exporting 4/26: 2_2_20220324T143731_20220324T145300_T18GYV.tif
Generating URL ...
Please wait ...
Data downloaded to /Users/gurupratap.matharu/Desktop/code/work/glaciers/sla-sca/exports/classifier/2_2_20220324T143731_20220324T145300_T